In [1]:
import sys 
import numpy as np
from resource import * 
import time
import psutil
import os 

In [2]:
def input_read(file):
    base_strings = []
    index_1 = []
    index_2 = []
    j=0
    k=0
    string_counter=0
    with open(file,"r") as f:
        for line in f.readlines():
            #remove all whitespaces in text file from reading in line by line 
            line=line.strip()
            # select only the base sequence to append to the sequence list 
            if not line.isnumeric():
                base_strings.append(line)
                #counter to make sure we are only appending strings not not the indices 
                string_counter+=1
            ##if the line is not a base sequence, then it is an index
            else:
                #if the string counter ==2, then that means are appending the second set of indices
                if string_counter == 2:
                    index_2.append(int(line))
                    k+=1
                #if the string !=2, then that means we are appending the first set of indices 
                else:
                    index_1.append(int(line))
                    j+=1
    return base_strings,index_1,index_2,k,j

In [3]:
def generate_sequences(base_strings,index_1,index_2,k,j):
    sequences=[]
    seq1=base_strings[0]
    seq2=base_strings[1]
    ##check if its string 1 or string 2 we are using 
    for i in index_1:
        seq1 = seq1[0:i+1] + seq1 + seq1[i+1:]
    if len(seq1) == (2**j)*len(base_strings[0]):
        sequences.append(seq1)
    for i in index_2:
        seq2 = seq2[0:i+1] + seq2 + seq2[i+1:]
    if len(seq2) == (2**k)*len(base_strings[1]):   
        sequences.append(seq2)
    return sequences

        

In [7]:
def mismatch_penalty(base_1,base_2):
    nucs="ACGT"
    index_1=nucs.index(base_1)
    index_2=nucs.index(base_2)
    mismatch_matrix = [[0,110,48,94],
                        [110,0,118,48],
                        [48,118,0,110],
                        [94,48,110,0]]
    return mismatch_matrix[index_1][index_2]

In [20]:
def alignment(seq1,seq2):
    # set value of gaps 
    gap_pen=30
    # build dp table
    m=len(seq1)
    n=len(seq2)
    dp = np.zeros([m+1,n+1], dtype=int)
    #initialize dp table 
    dp[0:(m+1),0] = [i*gap_pen for i in range(m+1)]
    dp[0,0:(n+1)] = [j*gap_pen for j in range(n+1)]
    #fill dp table with penalty scores 
    for i in range(1,m+1):
        for j in range(1,n+1):
            # if the nucs are equal to each other 
            if seq1[i-1] == seq2[j-1]:
                dp[i][j] = dp[i-1][j-1]
            # if the nucs don't match each other, then find min of the potential penalties 
            else:
                dp[i][j] = min(dp[i-1][j-1] + mismatch_penalty(seq1[i-1],seq2[j-1]),
                                dp[i-1][j] + gap_pen,
                                dp[i][j-1] + gap_pen)
    
    # find optimal path from dp table 
    i= m
    j= n
    #tracing path 
    seq1_list=[]
    seq2_list=[]
    while i!=0 or j!=0:
        #case 1: seq2 has gap 
        if dp[i][j]==dp[i-1][j]+gap_pen:
            seq1_list.append(seq1[i-1])
            seq2_list.append('_')
            i-=1
        #case 2: seq1 has gap 
        elif dp[i][j]==dp[i][j-1]+gap_pen:
            seq1_list.append('_')
            seq2_list.append(seq2[j-1])
            j-=1

        #case 3: nucs match, so no penalty or nucs mismatch, but penalty is already accounted for so same path 
        else:
            seq1_list.append(seq1[i-1])
            seq2_list.append(seq2[j-1])
            i-=1
            j-=1


    s1_final=''
    s2_final=''
    for k in range(len(seq1_list)):
        s1_final += seq1_list.pop()
        s2_final += seq2_list.pop()
    seq = [s1_final, s2_final]
    return dp[m][n],seq

In [6]:
inputs=input_read("../data/SampleTestCases/input1.txt")
test=generate_sequences(inputs[0],inputs[1],inputs[2],inputs[3],inputs[4])

In [22]:
txt = "Hello World" [::-1]
print(txt)

dlroW olleH


In [25]:
def dc_alignment(seq1,seq2):
    gap_pen=30 
    m=len(seq1)
    n=len(seq2)
    #if the length of the sequences is less than 2, then just use the dp algorithm as a bound 
    if n<=2 or m<=2:
        return alignment(seq1,seq2)
    f = np.zeros((m + 1, 2))
    g = np.zeros((m + 1, 2))

    for i in range(m + 1):
        f[i, 0] = i * gap_pen
    for j in range(1, n // 2 + 1):
        f[0, 1] = j * gap_pen
        for i in range(1, m + 1):
            f[i, 1] = min(f[i - 1, 0] + mismatch_penalty(seq1[i-1],seq2[j-1]),
                            f[i - 1, 1] + gap_pen,
                            f[i, 0] + gap_pen)
        for i in range(m+1):
            f[i,0]=f[i,1]
    
    seq1_inv=seq1[::-1]
    seq2_inv=seq2[::-1]
    for i in range(m + 1):
        g[i, 0] = i * gap_pen
    for j in range(1, n // 2 + 1):
        g[0, 1] = j * gap_pen
        for i in range(1, m + 1):
            g[i, 1] = min(g[i - 1, 0] + mismatch_penalty(seq1_inv[m-i],seq2_inv[n-j]),
                            g[i - 1, 1] + gap_pen,
                            g[i, 0] + gap_pen)
        for i in range(m+1):
            g[i,0]=f[i,1]
    q = 0
    for i in range(m + 1):
        if f[i, 0] + g[m - i, 0] < f[q, 0] + g[m - q, 0]:
            q = i
    # print(f[q, 0] + g[m - q, 0])
    alignment_x1, alignment_y1 = dc_alignment(seq1[:q], seq2[:n // 2])
    alignment_x2, alignment_y2 = dc_alignment(seq1[q:], seq2[n // 2:])
    return alignment_x1 + alignment_x2, alignment_y1 + alignment_y2
    
    

In [26]:
dc_alignment(test[0],test[1])

(1968,
 ['_A__',
  'TATT',
  'CA_',
  '_AT',
  'CA',
  'TA',
  '__C_',
  'TACG',
  '_T',
  'CT',
  'GACTA',
  '_A_T_',
  'C_T_',
  'TATA',
  '_G_A',
  'CGCG',
  '_C',
  'AC',
  'TG_',
  '_GC',
  'G_',
  'GG',
  'TGAC',
  '__AC',
  '__',
  'GC',
  '_TAC',
  'GT__',
  '_T',
  'AT',
  'GAC',
  '_AC',
  '_T',
  'GC',
  '_GGACT',
  'T__A__',
  '__',
  'TT',
  'GACTACT',
  '_A_T___',
  'GA_',
  '_AC',
  '_CTG',
  'GC__',
  'G_',
  'GA',
  'TGAC',
  'CG__',
  'T__A',
  'CGGA',
  'CTG',
  'C_G',
  'ACTGG',
  '_C_G_'])

In [14]:
output1[0]

'_A__CA_CA__C__TGACTAC_T__G_A_CTG_G_TGAC___TAC_TGAC_T_GGACT__GACTACTGA__CTGG_TGACT__ACTGACTGG'

In [17]:
mismatch=0
gap=0
for i in range(len(output1[0])):
    if output1[0][i]!=output1[1][i]:
        if output1[0][i]=='_' or output1[1][i]=='_':
            gap+=30
        else:
            mismatch+=mismatch_penalty(output1[0][i],output1[1][i])
score=mismatch+gap
print(score)
    

1968


In [21]:
m=len(test[0])
n=len(test[1])
gap_pen=30 
f = np.zeros((m + 1, 2))
g = np.zeros((m + 1, 2))
for i in range(m + 1):
        f[i, 0] = i * gap_pen

In [20]:
for j in range(1, n // 2 + 1):
    f[0, 1] = j * gap_pen
    for i in range(1, m + 1):
        f[i, 1] = min(f[i - 1, 0] + mismatch_penalty[i-1][j-1],
                          f[i - 1, 1] + gap_pen,
                          f[i, 0] + gap_pen)

2